In [52]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
pd.set_option('display.max_columns', None)
#from sqlalchemy import create_engine
import sqlite3


In [59]:
def fetch_data(web_driver, keyword, y, m, da, filter=None):
    try:
        # Construct the URL
        # Base URL construction
        bet_search = f'https://www.nba.com/stats/players/{keyword}?DateFrom={m}%2F{da}%2F{y}&DateTo={m}%2F{da}%2F{y}&SeasonType=Regular+Season'
        
        # Add filter to URL if provided
        if filter:
            bet_search += f'&{filter}'

        
        web_driver.get(bet_search)

        # Wait for the page to load
        WebDriverWait(web_driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[3]/section[1]/div/div/div[2]/label/div/select")))

        """# Select "Regular Season"
        try:
            select_element = WebDriverWait(web_driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[3]/section[1]/div/div/div[2]/label/div/select"))
            )
            select_element.click()
            season_select = Select(select_element)
            time.sleep(2)
            season_select.select_by_visible_text("Regular Season")
        except (NoSuchElementException, TimeoutException):
            print("Could not select 'Regular Season', continuing...")"""

        # Select "All" from second dropdown
        try:
            select_element = WebDriverWait(web_driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
            )
            select_element.click()
            all_select = Select(select_element)
            time.sleep(2)
            all_select.select_by_visible_text("All")
        except (NoSuchElementException, TimeoutException):
            print("Could not select 'All', continuing...")

        # Parse the page and extract the table
        soup = BeautifulSoup(web_driver.page_source, 'html.parser')
        tab = soup.find("table", {"class": "Crom_table__p1iZz"})
        if tab:
            df = pd.read_html(str(tab))[0]
            df['GAME_DATE'] = f"{y}-{m}-{da}"
            return df
        else:
            print("No table found.")
            return None

    except NoSuchElementException as e:
        print(f"Element not found: {e}")
        return None

    except ValueError as e:
        if str(e) == "No tables found":
            print("No tables found on the page.")
            return None
        else:
            raise e

In [54]:

from nba_api.stats.endpoints import leaguegamefinder
#import pandas as pd

# Define the parameters (optional: specify dates or team)
schedule = leaguegamefinder.LeagueGameFinder(season_nullable='2023-24', season_type_nullable='Regular Season').get_data_frames()[0]

# View schedule DataFrame
#print(schedule.head())

# Filter columns if needed
filtered_schedule = pd.DataFrame(schedule[['GAME_DATE','TEAM_ABBREVIATION', 'MATCHUP', 'SEASON_ID']])

In [55]:
filtered_schedule[['Left', 'Right']] = filtered_schedule['MATCHUP'].str.split(r' vs\. | @ ', expand=True)
filtered_schedule.rename(columns={'Left':'TEAM','Right':'Opponent'}, inplace=True)

In [57]:
filtered_schedule['GAME_DATE'] = pd.to_datetime(filtered_schedule['GAME_DATE'])

# Extract Year, Month, and Day into new columns
filtered_schedule['Year'] = filtered_schedule['GAME_DATE'].dt.year
filtered_schedule['Month'] = filtered_schedule['GAME_DATE'].dt.month
filtered_schedule['Day'] = filtered_schedule['GAME_DATE'].dt.day

In [58]:
conn = sqlite3.connect('nba_data.db')

In [50]:
# NEED TO GET SCHEDULE FOR DATES
#dates = pd.DataFrame([[2024, '02', '01'], [2024, '02', '02'], [2024, '02', '03']], columns=['Year', 'Month', 'Day'])
keywords = ['drives', 'passing', 'rebounding','catch-shoot','speed-distance','pullup']  # Add your desired keywords here

# Initialize WebDriver
web_driver = webdriver.Safari()

for keyword in keywords:
    keyword_data = []  # Temporary list to store DataFrames for the current keyword
    for i, d in filtered_schedule.iterrows():
        y, m, da = d['Year'], d['Month'], d['Day']
        try:
            df = fetch_data(web_driver, keyword, y, m, da)
            time.sleep(4)
            if df is not None:
                keyword_data.append(df)
        except Exception as e:
            print(f"An error occurred for {keyword} on {y}-{m}-{da}: {e}")

    # Combine each keyword's data into a single DataFrame and assign it a dynamic name
    globals()[f"{keyword}_df"] = pd.concat(keyword_data, ignore_index=True) if keyword_data else pd.DataFrame()

# Clean up and close the WebDriver
web_driver.quit()

In [61]:
# NEED TO GET SCHEDULE FOR DATES
dates = pd.DataFrame([[2024, '02', '01'], [2024, '02', '02'], [2024, '02', '03']], columns=['Year', 'Month', 'Day'])
keywords_with_filters = {
    'shots-shotclock': ["ShotClockRange=24-22", "ShotClockRange=22-18+Very+Early",'ShotClockRange=18-15+Early','ShotClockRange=15-7+Average','ShotClockRange=7-4+Late','ShotClockRange=4-0+Very+Late'],  # Filter options for 'shots-shotclock'
    'shots-dribbles': ["DribbleRange=0+Dribbles", "DribbleRange=1+Dribble",'DribbleRange=2+Dribbles','DribbleRange=3-6+Dribbles','DribbleRange=7%2B+Dribbles']   # Filter options for 'passing'
} # Add your desired keywords here

# Initialize WebDriver
web_driver = webdriver.Safari()

for keyword, filters in keywords_with_filters.items():
    keyword_data = []  # Temporary list to store DataFrames for the current keyword
    for i, d in dates.iterrows():
        y, m, da = d['Year'], d['Month'], d['Day']
        for filter_option in filters:
            try:
                # Fetch data with the current filter
                df = fetch_data(web_driver, keyword, y, m, da, filter=filter_option)
                #time.sleep(2)
                if df is not None:
                    df[f'{keyword}'] = filter_option  # Add filter to the DataFrame for reference
                    keyword_data.append(df)
            except Exception as e:
                print(f"An error occurred for {keyword} with filter {filter_option} on {y}-{m}-{da}: {e}")

    # Combine each keyword's data into a single DataFrame and assign it a dynamic name
    globals()[f"{keyword}_df"] = pd.concat(keyword_data, ignore_index=True) if keyword_data else pd.DataFrame()

# Clean up and close the WebDriver
web_driver.quit()

Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...
Could not select 'All', continuing...


In [65]:
shots_shotclock_df = globals().get("shots-shotclock_df")

In [68]:
shots_shotclock_df

,0_PLAYER,1_TEAM,2_AGE,3_GP,4_G,5_FREQ%,6_FGM,7_FGA,8_FG%,9_EFG%,Field Goals_2FG FREQ%,Field Goals_2FGM,Field Goals_2FGA,Field Goals_2FG%,Field Goals_3FG FREQ%,2 Point Field Goals_3PM,2 Point Field Goals_3PA,2 Point Field Goals_3P%,GAME_DATE,shots-shotclock
0,Isaiah Hartenstein,NYK,26,1,1,50.0,3.0,4.0,75.0,75.0,50.0,3.0,4.0,75.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
1,Precious Achiuwa,NYK,24,1,1,18.8,2.0,3.0,66.7,66.7,18.8,2.0,3.0,66.7,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
2,Neemias Queta,BOS,24,1,1,40.0,1.0,2.0,50.0,50.0,40.0,1.0,2.0,50.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
3,Pascal Siakam,IND,30,1,1,10.5,1.0,2.0,50.0,50.0,10.5,1.0,2.0,50.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
4,Jayson Tatum,BOS,26,1,1,9.5,0.0,2.0,0.0,0.0,9.5,0.0,2.0,0.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,Max Strus,CLE,28,1,1,11.1,0.0,1.0,0.0,0.0,11.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1366,Clint Capela,ATL,30,1,1,9.1,0.0,1.0,0.0,0.0,9.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1367,Keldon Johnson,SAS,24,1,1,9.1,0.0,1.0,0.0,0.0,9.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1368,DeMar DeRozan,CHI,34,1,1,7.1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-,7.1,0.0,1.0,0.0,2024-02-03,ShotClockRange=4-0+Very+Late


In [69]:
if isinstance(shots_shotclock_df.columns, pd.MultiIndex):
    shots_shotclock_df.columns = ['_'.join(filter(None, col)).strip() for col in shots_shotclock_df.columns]

# Step 2: Rename columns to remove "Unnamed" text
shots_shotclock_df.columns = [col.replace("Unnamed: ", "").replace("_level_0", "").strip() for col in shots_shotclock_df.columns]

# Step 3: Drop empty columns
shots_shotclock_df = shots_shotclock_df.dropna(axis=1, how='all')
shots_shotclock_df.columns = shots_shotclock_df.columns.str.replace(r'^\d+_', '', regex=True)
shots_shotclock_df

,0_PLAYER,1_TEAM,2_AGE,3_GP,4_G,5_FREQ%,6_FGM,7_FGA,8_FG%,9_EFG%,Field Goals_2FG FREQ%,Field Goals_2FGM,Field Goals_2FGA,Field Goals_2FG%,Field Goals_3FG FREQ%,2 Point Field Goals_3PM,2 Point Field Goals_3PA,2 Point Field Goals_3P%,GAME_DATE,shots-shotclock
0,Isaiah Hartenstein,NYK,26,1,1,50.0,3.0,4.0,75.0,75.0,50.0,3.0,4.0,75.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
1,Precious Achiuwa,NYK,24,1,1,18.8,2.0,3.0,66.7,66.7,18.8,2.0,3.0,66.7,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
2,Neemias Queta,BOS,24,1,1,40.0,1.0,2.0,50.0,50.0,40.0,1.0,2.0,50.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
3,Pascal Siakam,IND,30,1,1,10.5,1.0,2.0,50.0,50.0,10.5,1.0,2.0,50.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
4,Jayson Tatum,BOS,26,1,1,9.5,0.0,2.0,0.0,0.0,9.5,0.0,2.0,0.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,Max Strus,CLE,28,1,1,11.1,0.0,1.0,0.0,0.0,11.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1366,Clint Capela,ATL,30,1,1,9.1,0.0,1.0,0.0,0.0,9.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1367,Keldon Johnson,SAS,24,1,1,9.1,0.0,1.0,0.0,0.0,9.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1368,DeMar DeRozan,CHI,34,1,1,7.1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-,7.1,0.0,1.0,0.0,2024-02-03,ShotClockRange=4-0+Very+Late


,PLAYER,TEAM,AGE,GP,G,FREQ%,FGM,FGA,FG%,EFG%,Field Goals_2FG FREQ%,Field Goals_2FGM,Field Goals_2FGA,Field Goals_2FG%,Field Goals_3FG FREQ%,2 Point Field Goals_3PM,2 Point Field Goals_3PA,2 Point Field Goals_3P%,GAME_DATE,shots-shotclock
0,Isaiah Hartenstein,NYK,26,1,1,50.0,3.0,4.0,75.0,75.0,50.0,3.0,4.0,75.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
1,Precious Achiuwa,NYK,24,1,1,18.8,2.0,3.0,66.7,66.7,18.8,2.0,3.0,66.7,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
2,Neemias Queta,BOS,24,1,1,40.0,1.0,2.0,50.0,50.0,40.0,1.0,2.0,50.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
3,Pascal Siakam,IND,30,1,1,10.5,1.0,2.0,50.0,50.0,10.5,1.0,2.0,50.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
4,Jayson Tatum,BOS,26,1,1,9.5,0.0,2.0,0.0,0.0,9.5,0.0,2.0,0.0,0.0,0.0,0.0,-,2024-02-01,ShotClockRange=24-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,Max Strus,CLE,28,1,1,11.1,0.0,1.0,0.0,0.0,11.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1366,Clint Capela,ATL,30,1,1,9.1,0.0,1.0,0.0,0.0,9.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1367,Keldon Johnson,SAS,24,1,1,9.1,0.0,1.0,0.0,0.0,9.1,0.0,1.0,0.0,0.0,0.0,0.0,-,2024-02-03,ShotClockRange=4-0+Very+Late
1368,DeMar DeRozan,CHI,34,1,1,7.1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-,7.1,0.0,1.0,0.0,2024-02-03,ShotClockRange=4-0+Very+Late


In [51]:
# For each keyword
for keyword in keywords:
    existing_data = pd.read_sql(f"SELECT * FROM {keyword}_data", conn)
    new_data = pd.concat(keyword_data, ignore_index=True) if keyword_data else pd.DataFrame()
    
    # Check for new rows
    if not new_data.empty:
        combined = pd.concat([existing_data, new_data]).drop_duplicates(subset=['GAME_DATE', 'PLAYER_ID'], keep='first')
        new_rows = combined[~combined.index.isin(existing_data.index)]
        
        # Append only new rows
        new_rows.to_sql(f"{keyword}_data", conn, if_exists="append", index=False)

,PLAYER,TEAM,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,FTM,FTA,FT%,PTS,PTS%,PASS,PASS%,AST,AST%,TO,TOV%,PF,PF%,GAME_DATE
0,Aaron Nesmith,IND,1,0,1,36.6,7.0,2.0,5.0,40.0,0.0,0.0,0.0,4.0,57.1,0.0,0.0,0.0,0.0,1.0,14.3,0.0,0.0,2024-02-01
1,Al Horford,BOS,1,0,1,23.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-02-01
2,Andrew Nembhard,IND,1,0,1,26.1,11.0,3.0,8.0,37.5,0.0,0.0,0.0,6.0,54.5,3.0,27.3,1.0,9.1,0.0,0.0,0.0,0.0,2024-02-01
3,Austin Reaves,LAL,1,1,0,34.6,11.0,1.0,4.0,25.0,0.0,0.0,0.0,2.0,18.2,7.0,63.6,3.0,27.3,0.0,0.0,0.0,0.0,2024-02-01
4,Ben Sheppard,IND,1,0,1,22.1,1.0,1.0,1.0,100.0,0.0,0.0,0.0,2.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,Trendon Watford,BKN,1,1,0,8.6,2.0,1.0,2.0,50.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-02-03
411,Trey Lyles,SAC,1,1,0,24.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1.0,50.0,0.0,0.0,0.0,0.0,2024-02-03
412,Tyrese Maxey,PHI,1,0,1,30.6,17.0,3.0,13.0,23.1,4.0,4.0,100.0,10.0,58.8,1.0,5.9,0.0,0.0,0.0,0.0,2.0,11.8,2024-02-03
413,Victor Wembanyama,SAS,1,0,1,26.9,9.0,0.0,2.0,0.0,1.0,2.0,50.0,1.0,11.1,5.0,55.6,1.0,11.1,1.0,11.1,1.0,11.1,2024-02-03
